In [1]:
# keras相关包
from tensorflow.keras.applications import ResNet50
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

import random
# sklearn相关包
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.image import extract_patches_2d

import numpy as np
import cv2
import os
from imutils import paths
import progressbar

# 作图
import matplotlib.pyplot as plt
%matplotlib inline

#Basic Tool
import pandas as pd
import sys
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

#设置随机种子
np.random.seed(161)

from keras.datasets import cifar10

In [4]:
#读取数据集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#归一化
x_train = x_train / 255.0
x_test = x_test / 255.0

170508288/170498071 [==============================] - 2s 0us/step


In [5]:
def imbalanceData(__format,X_train,y_train):
   """
   __format：The imbalance want to input
   X_train:Train Set
   y_train:Test Set
   """
   train_classes, train_class_counts=np.unique(y_train, return_counts=True)
   nb_train_classes = len(train_classes)
   class_indices = [np.where(y_train == i)[0] for i in range(nb_train_classes)]
   imbal_class_indices = [class_idx[:class_count] for class_idx, class_count in zip(class_indices, __format)]
   imbal_class_indices = np.hstack(imbal_class_indices)
   y = y_train[imbal_class_indices]
   X = x_train[imbal_class_indices]  
   return X,y

In [19]:
imbal_class_counts=[5000,5000]*5 #you can use the format such:[500,1000]*5 [100,200,300,400,500,...,1000]
x_train,y_train=imbalanceData(imbal_class_counts,x_train,y_train)

In [20]:
# Y 热编码
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

In [22]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,BatchNormalization

In [36]:
#定义模型
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=x_train.shape[1:],activation='relu'))
model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 8, 8, 64)         

In [37]:
#训练模型
print('-----------Training-----------')
model.fit(x_train,y_train_oh,batch_size=32,epochs=20,verbose=2)

#评估模型
print('-----------Testing-------------')
scores=model.evaluate(x_test,y_test_oh,verbose=0)
print('loss:%s,accuracy:%s' % (scores[0],scores[1]))

# #模型保存
# model.save('cifar10_cnn_model.h5')

-----------Training-----------
Epoch 1/20
860/860 - 9s - loss: 1.2709 - accuracy: 0.5196
Epoch 2/20
860/860 - 8s - loss: 0.9339 - accuracy: 0.6591
Epoch 3/20
860/860 - 8s - loss: 0.7724 - accuracy: 0.7164
Epoch 4/20
860/860 - 8s - loss: 0.6665 - accuracy: 0.7604
Epoch 5/20
860/860 - 8s - loss: 0.5806 - accuracy: 0.7859
Epoch 6/20
860/860 - 8s - loss: 0.5055 - accuracy: 0.8133
Epoch 7/20
860/860 - 8s - loss: 0.4508 - accuracy: 0.8339
Epoch 8/20
860/860 - 8s - loss: 0.3889 - accuracy: 0.8579
Epoch 9/20
860/860 - 8s - loss: 0.3406 - accuracy: 0.8764
Epoch 10/20
860/860 - 8s - loss: 0.3105 - accuracy: 0.8891
Epoch 11/20
860/860 - 8s - loss: 0.2779 - accuracy: 0.8974
Epoch 12/20
860/860 - 8s - loss: 0.2401 - accuracy: 0.9117
Epoch 13/20
860/860 - 8s - loss: 0.2337 - accuracy: 0.9149
Epoch 14/20
860/860 - 8s - loss: 0.2106 - accuracy: 0.9240
Epoch 15/20
860/860 - 8s - loss: 0.1946 - accuracy: 0.9305
Epoch 16/20
860/860 - 8s - loss: 0.1911 - accuracy: 0.9327
Epoch 17/20
860/860 - 8s - loss: 0

In [38]:
preds = model.predict(x_test, batch_size=32)
print(classification_report(y_test_oh.argmax(axis=1),preds.argmax(axis=1), digits=4))

              precision    recall  f1-score   support

           0     0.7146    0.5910    0.6470      1000
           1     0.7156    0.9210    0.8054      1000
           2     0.7029    0.2390    0.3567      1000
           3     0.3412    0.7230    0.4636      1000
           4     0.7432    0.1910    0.3039      1000
           5     0.5101    0.7050    0.5919      1000
           6     0.9007    0.3990    0.5530      1000
           7     0.6061    0.8680    0.7138      1000
           8     0.8444    0.5860    0.6919      1000
           9     0.7211    0.8790    0.7922      1000

    accuracy                         0.6102     10000
   macro avg     0.6800    0.6102    0.5919     10000
weighted avg     0.6800    0.6102    0.5919     10000

